In [ ]:
# ORCID Member clientID and secret
ID=""
SEC=""

In [ ]:
import requests

Get an Access token
```
URL=https://orcid.org/oauth/token
  HEADER: Accept: application/json
  METHOD: POST
  DATA: 
    client_id=[Your public API client ID]
    client_secret=[Your public API secret]
    grant_type=client_credentials
    scope=/read-public
```

In [ ]:
URL="https://orcid.org/oauth/token"
HEADERS={"Accept": "application/json"}
METHOD="POST"
DATA={"client_id": ID, "client_secret": SEC,  "grant_type": "client_credentials", "scope": "/read-public"}


In [ ]:
res=requests.post(url=URL, headers=HEADERS, data=DATA)

In [ ]:
res

In [ ]:
bearer = res.json()['access_token']

Info to query employments info in ORCID
```
Method: GET
Headers:
  Accept: application/json
  Authorization: Bearer [Stored access token]
URL: https://api.orcid.org/v3.0/[ORCID iD]/employments
```

In [ ]:
file_path = 'orcids.csv'
orcids = []

# Reading all lines from the file
try:
    with open(file_path, 'r') as file:
        lines = file.readlines()  # Read all lines into a list
        for line in lines:
            orcids.append(line.strip())  # Print each line, stripping newline characters
except FileNotFoundError:
    print(f"The file at {file_path} was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

In [34]:
len(orcids)

970

In [ ]:
orgs = []
for orcid in orcids:
    headers = {"Accept": "application/json", "Authorization": "Bearer %s" % bearer}
    url = "https://api.orcid.org/v3.0/%s/employments" % orcid
    orc = requests.get(url=url, headers=headers)
    data=orc.json() 
    organization_names = []

    # Loop through 'affiliation-group'
    for group in data.get('affiliation-group', []):
        # Loop through 'summaries' within each group
        for summary in group.get('summaries', []):
            # Extract 'employment-summary' and then the organization name
            employment_summary = summary.get('employment-summary', {})
            organization = employment_summary.get('organization', {})
            organization_name = organization.get('name', None)
            if organization_name:  # Add to the list if not None
                organization_names.append(organization_name)

    print(f"{orcid}: {organization_names}")
    orgs.append({orcid: organization_names})


In [35]:
len(orgs) # should match length of orcid list

970

In [ ]:
import json

In [ ]:
with open("orcid-orgs.json", 'w') as file:
        json.dump(orgs, file, indent=4)